# Tratamento de Dados

### Bibliotecas

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import factor_analyzer

### Limpeza do Dicionário


Carregamento e formatação do dicionário de dados, removendo espaços em branco nos nomes das colunas e eliminando linhas sem identificação de variáveis para garantir a correta consulta das descrições.

In [2]:
path_dic= r"../dados/microdados_enem_2019/DICIONÁRIO/Dicionário_Microdados_Enem_2019.xlsx"
path_microdados=r"../dados/microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv"

In [ ]:
dic_enem = pd.read_excel(path_dic, header=1)


dic_enem.columns = dic_enem.columns.str.strip()


dic_enem.rename(columns={dic_enem.columns[0]: 'NOME DA VARIÁVEL'}, inplace=True)


dic_enem = dic_enem.dropna(subset=['NOME DA VARIÁVEL'])

### Carregamento em Chunks

Leitura dos microdados do ENEM utilizando o parâmetro chunksize. Optamos por essa estratégia para processar arquivos massivos que excedem a memória RAM disponível, permitindo a concatenação progressiva dos dados.

**Observação:** A célula abaixo não irá executar, pois a variável 'path_microdados' foi apagada da base por conta do tamanho excesivo. Já que só utilizamos ela nesta parte, não vimos sentido em deixar um arquivo tão pesado.

In [ ]:
df_participantes_prova=pd.read_csv(path_microdados, encoding='latin-1', sep=";",chunksize=1e5, low_memory=False)

chunks_processados=[] 
for i, chunk in enumerate(df_participantes_prova):
  chunks_processados.append(chunk)
  df_semi_final= pd.concat(chunks_processados)

### Remoção de Colunas Irrelevantes

Eliminação de variáveis que não serão utilizadas na modelagem estatística, visando reduzir a dimensionalidade do dataset. Como queremos fazer a análise baseando-nos na nota final, dados como ano da prova e cor da prova são irrelevantes para isso, vamos tirar essas colunas.

In [ ]:
colunas_para_remover = ['NU_ANO','TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT','TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT','NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC','CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA']
df_semi_final.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

### Filtragem de Treineiros

Remoção dos candidatos inscritos apenas para fins de treinamento. O foco da análise é mantido apenas nos alunos que efetivamente concorreram a vagas no ensino superior.

In [ ]:
df_semi_final= df_semi_final[df_semi_final['IN_TREINEIRO']==0]
df_semi_final.drop(columns=['IN_TREINEIRO'], inplace=True)

### Inspeção de Metadados

Esse passo serve para apenas validar o significado de cada variável socioeconômica antes das transformações numéricas.

In [ ]:
colunas_descricao =['NU_INSCRICAO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL',
       'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU',
       'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'CO_MUNICIPIO_ESC',
       'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA',
       'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA', 'TP_PRESENCA_CN',
       'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'NU_NOTA_CN',
       'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA',
       'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',
       'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013',
       'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
       'Q023', 'Q024', 'Q025']
descricoes = dic_enem[dic_enem['NOME DA VARIÁVEL'].isin(colunas_descricao)]

descricoes.columns = descricoes.columns.str.strip()


print("-" * 30)
print("DICIONÁRIO DE VARIÁVEIS SELECIONADAS")
print("-" * 30 + "\n")


for index, row in descricoes.iterrows():
    var_nome = row['NOME DA VARIÁVEL']
    var_desc = row['Descrição'] if 'Descrição' in descricoes.columns else row.iloc[1]
    print(f" {var_nome}:")
    print(f"{var_desc}\n")

------------------------------
DICIONÁRIO DE VARIÁVEIS SELECIONADAS
------------------------------

 NU_INSCRICAO:
Número de inscrição1 

 TP_FAIXA_ETARIA:
Faixa etária2

 TP_SEXO:
Sexo

 TP_ESTADO_CIVIL:
Estado Civil

 TP_COR_RACA:
Cor/raça

 TP_NACIONALIDADE:
Nacionalidade

 TP_ST_CONCLUSAO:
Situação de conclusão do Ensino Médio

 TP_ANO_CONCLUIU:
Ano de Conclusão do Ensino Médio

 TP_ESCOLA:
Tipo de escola do Ensino Médio

 TP_ENSINO:
Tipo de instituição que concluiu ou concluirá o Ensino Médio 

 IN_TREINEIRO:
Indica se o inscrito fez a prova com intuito de apenas treinar seus conhecimentos3

 NO_MUNICIPIO_ESC:
Nome do município da escola

 CO_UF_ESC:
Código da Unidade da Federação da escola

 SG_UF_ESC:
Sigla da Unidade da Federação da escola

 TP_DEPENDENCIA_ADM_ESC:
Dependência administrativa (Escola)

 TP_LOCALIZACAO_ESC:
Localização (Escola)

 TP_SIT_FUNC_ESC:
Situação de funcionamento (Escola)

 CO_MUNICIPIO_PROVA:
Código do município da aplicação da prova

 NO_MUNICIPIO_PROV

Após tirar os treineiros e fazer o pré-processamento de colunas que não servem ao modelo, ainda nos restam mais de 4 milhões de linhas. Para todos os efeitos, essa base de dados é muito grande para nossa tarefa.

In [10]:
for column in df_semi_final.columns:
  print(f'{column}: {df_semi_final[column].unique()}')

NU_INSCRICAO: [190001595656 190001421546 190001133210 ... 190006087652 190006087653
 190006087654]
TP_FAIXA_ETARIA: [13  8 10  7  5  2  6  4 11  3 12 14 15 20  9 16 17  1 18 19]
TP_SEXO: ['M' 'F']
TP_ESTADO_CIVIL: [1 2 0 3 4]
TP_COR_RACA: [3 1 2 4 0 5]
TP_NACIONALIDADE: [1 2 4 3 0]
TP_ST_CONCLUSAO: [1 2 4]
TP_ANO_CONCLUIU: [11  5 13  4  1  0  6  2  9 10  8  7  3 12]
TP_ESCOLA: [1 2 3]
TP_ENSINO: [nan  1.  2.]
CO_UF_ESC: [nan 31. 41. 43. 35. 29. 25. 13. 53. 23. 22. 15. 33. 24. 50. 51. 17. 27.
 52. 16. 26. 42. 32. 21. 11. 12. 28. 14.]
SG_UF_ESC: [nan 'MG' 'PR' 'RS' 'SP' 'BA' 'PB' 'AM' 'DF' 'CE' 'PI' 'PA' 'RJ' 'RN' 'MS'
 'MT' 'TO' 'AL' 'GO' 'AP' 'PE' 'SC' 'ES' 'MA' 'RO' 'AC' 'SE' 'RR']
TP_DEPENDENCIA_ADM_ESC: [nan  2.  4.  1.  3.]
TP_LOCALIZACAO_ESC: [nan  1.  2.]
TP_SIT_FUNC_ESC: [nan  1.  4.  2.  3.]
TP_PRESENCA_CN: [0 1 2]
TP_PRESENCA_CH: [0 1 2]
TP_PRESENCA_LC: [0 1 2]
TP_PRESENCA_MT: [0 1 2]
NU_NOTA_CN: [  nan 483.8 513.6 ... 796.7 814.8 788.3]
NU_NOTA_CH: [  nan 503.6 575.5 ... 775.

### Filtro de Presença Total

Vamos escolher somente quem concluiu a prova como um todo sem problemas

In [ ]:
df_semi_final= df_semi_final[(df_semi_final['TP_PRESENCA_CN']==1) & (df_semi_final['TP_PRESENCA_CH']==1) & (df_semi_final['TP_PRESENCA_LC']==1) & (df_semi_final['TP_PRESENCA_MT']==1)&(df_semi_final['TP_STATUS_REDACAO']==1)]

### Limpeza de Variáveis de Controle

Remoção das colunas de presença e de língua estrangeira após a filtragem, uma vez que elas perdem a utilidade após a seleção dos candidatos presentes.

In [13]:
df_semi_final.drop(columns=['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT','TP_STATUS_REDACAO','TP_LINGUA' ], inplace=True)

In [15]:
for nan in df_semi_final.columns:
  print(f'{nan}: {df_semi_final[nan].isna().sum()}')

NU_INSCRICAO: 0
TP_FAIXA_ETARIA: 0
TP_SEXO: 0
TP_ESTADO_CIVIL: 0
TP_COR_RACA: 0
TP_NACIONALIDADE: 0
TP_ST_CONCLUSAO: 0
TP_ANO_CONCLUIU: 0
TP_ESCOLA: 0
TP_ENSINO: 1024663
CO_UF_ESC: 2168215
SG_UF_ESC: 2168215
TP_DEPENDENCIA_ADM_ESC: 2168215
TP_LOCALIZACAO_ESC: 2168215
TP_SIT_FUNC_ESC: 2168215
NU_NOTA_CN: 0
NU_NOTA_CH: 0
NU_NOTA_LC: 0
NU_NOTA_MT: 0
NU_NOTA_REDACAO: 0
Q001: 0
Q002: 0
Q003: 0
Q004: 0
Q005: 0
Q006: 0
Q007: 0
Q008: 0
Q009: 0
Q010: 0
Q011: 0
Q012: 0
Q013: 0
Q014: 0
Q015: 0
Q016: 0
Q017: 0
Q018: 0
Q019: 0
Q020: 0
Q021: 0
Q022: 0
Q023: 0
Q024: 0
Q025: 0


### Tratamento de Valores Ausentes

Pelo que pesquisamos, inscritos que não estão no ano de conclusão do Ensino Médio podem optar por não responder o questionário sobre escolaridade. Como nosso objetivo é tomar indivíduos saindo do Ensino Médio, vamos simplesmente retirar essas linhas.

In [ ]:
#droppando os nan's de TP_ENSINO e  CO_UF_ESC
df_semi_final.dropna(subset=['TP_ENSINO', 'CO_UF_ESC'], inplace=True)

In [ ]:
df_semi_final['MEDIA_CANDIDATO']=(df_semi_final['NU_NOTA_CN']+df_semi_final['NU_NOTA_CH']+df_semi_final['NU_NOTA_LC']+df_semi_final['NU_NOTA_MT']+df_semi_final['NU_NOTA_REDACAO'])/5

### Remoção de Notas Individuais

Exclusão das notas brutas das provas objetivas e da redação, tendo em vista mais uma vez que queremos apenas a nota final resultante.

In [18]:
df_semi_final.drop(columns=['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO'], inplace=True)

### Engenharia de Atributos Socioeconômicos

Tomando o critério Brasil mencionado em https://abep.org/criterio-brasil/ para o ano de 2019, temos:

- SCORE_CONSUMO: Soma ponderada de bens materiais.
- SCORE_CULT_PAIS: Nível de instrução agregado dos pais.
- RENDA: Conversão de faixas categóricas em valores monetários (pontos médios).
- INTERNET: Transformação da posse de internet em variável binária.

In [21]:
pesos_abep = {
    'Q008': {'A': 0, 'B': 3, 'C': 7, 'D': 10, 'E': 14}, # Banheiros
    'Q007': {'A': 0, 'B': 3, 'C': 7, 'D': 10, 'E': 13}, # Empregados domésticos
    'Q010': {'A': 0, 'B': 3, 'C': 5, 'D': 8, 'E': 11},  # Automóveis
    'Q024': {'A': 0, 'B': 3, 'C': 6, 'D': 8, 'E': 11},  # Microcomputador
    'Q017': {'A': 0, 'B': 3, 'C': 6, 'D': 6, 'E': 6},   # Lava louça
    'Q012': {'A': 0, 'B': 2, 'C': 3, 'D': 5, 'E': 5},   # Geladeira
    'Q013': {'A': 0, 'B': 2, 'C': 4, 'D': 6, 'E': 6},   # Freezer
    'Q014': {'A': 0, 'B': 2, 'C': 4, 'D': 6, 'E': 6},   # Lava roupa
    'Q020': {'A': 0, 'B': 1, 'C': 3, 'D': 4, 'E': 6},   # DVD
    'Q016': {'A': 0, 'B': 2, 'C': 4, 'D': 4, 'E': 4},   # Micro-ondas
    'Q011': {'A': 0, 'B': 1, 'C': 3, 'D': 3, 'E': 3},   # Motocicleta
    'Q015': {'A': 0, 'B': 2, 'C': 2, 'D': 2, 'E': 2}    # Secadora roupa
}
 #pesos segundo a abep para cada questão do questionário socioeconômico, vide dicionário do enem para mais detalhes
mapa_instrucao = {
'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 4, 'F': 7, 'G': 7, 'H': np.nan
}
mapa_renda_valores = {
'A': 0, 'B': 499, 'C': 1247, 'D': 1746, 'E': 2245, 'F': 2744,
'G': 3493, 'H': 4491, 'I': 5489, 'J': 6487, 'K': 7485, 'L': 8483,
'M': 9481, 'N': 10978, 'O': 13473, 'P': 17465, 'Q': 25000
} #pontos médios de cada faixa de renda


### Simplificação do Questionário

Remoção de perguntas socioeconômicas redundantes ou de menor impacto para facilitar a interpretação do número de pessoas residentes no domicílio.

In [ ]:
def resumir_socioeconomico_enem(df):
    df_proc = df.copy()
    
    #  SCORE_CONSUMO -> serve pra dar uma dimensão da renda não afirmada do candidato
    df_proc['SCORE_CONSUMO'] = 0
    for col, mapping in pesos_abep.items():
        if col in df_proc.columns:
            df_proc['SCORE_CONSUMO'] += df_proc[col].map(mapping).fillna(0)
            
    #  SCORE_CULT_PAIS (soma da instrução do pai e da mãe)
    if 'Q001' in df_proc.columns and 'Q002' in df_proc.columns:
        pai = df_proc['Q001'].map(mapa_instrucao).fillna(0)
        mae = df_proc['Q002'].map(mapa_instrucao).fillna(0)
        df_proc['SCORE_CULT_PAIS'] = pai + mae
        
    #  MAPA_RENDA
    if 'Q006' in df_proc.columns:
        df_proc['RENDA'] = df_proc['Q006'].map(mapa_renda_valores).fillna(0)
        
    #  Renomear Q025 para INTERNET
    if 'Q025' in df_proc.columns:
        df_proc = df_proc.rename(columns={'Q025': 'INTERNET'})
        df_proc['INTERNET'] = df_proc['INTERNET'].map({'A': 0, 'B': 1}).fillna(0)
        
    #  Dropar colunas que foram resumidas
    cols_to_drop = list(pesos_abep.keys()) + ['Q001', 'Q002', 'Q006']
    df_proc = df_proc.drop(columns=[c for c in cols_to_drop if c in df_proc.columns])
    
    return df_proc


In [ ]:
df_semi_final_numerico= resumir_socioeconomico_enem(df_semi_final)

Já que temos uma medida pra renda informada e consumo, temos bastante informação sobre o status socieconômico do candidato. Portanto, vamos retirar colunas de perguntas sobre objetos de consumo sobre as quais não encontramos um peso oficial na pontuação do objeto. 

Para facilitar a análise, também retiramos a coluna sobre a profissão dos pais, já que por serem de categorias muito extensas se torna pouco informativo, e usaremos o grau de instrução como aproximação para isso. 

Manteneremos a Q005 pois ela já é numérica e, portanto, não é necessário estipular nenhum peso como para as outras.

In [ ]:
subset=['Q003', 'Q004','Q009', 'Q018', 'Q019', 'Q021', 'Q022', 'Q023' ]

df_semi_final_numerico.rename(columns={'Q005': 'N_PESSOAS_MESMA_RED'}, inplace=True) #Número de pessoas que moram na mesma residência do candidato  

df_semi_final_numerico.drop(columns=subset, inplace=True)

display(df_semi_final_numerico.columns)


### Codificação de Sexo

Transformação da variável categórica de sexo em numérica (binária), facilitando a aplicação de modelos estatísticos e cálculos de correlação.

In [25]:
mapa_sexo={'M':0,'F':1}
df_semi_final_numerico['TP_SEXO']=df_semi_final_numerico['TP_SEXO'].map(mapa_sexo)

In [ ]:
df_semi_final_numerico.set_index('NU_INSCRICAO', inplace=True)

### Mapeamento Geográfico

Conversão das siglas dos estados em identificadores numéricos e remoção de códigos redundantes, finalizando a preparação das variáveis de localização para o modelo.

In [ ]:
estados=df_semi_final_numerico['SG_UF_ESC'].unique()

array(['PR', 'RS', 'SP', 'BA', 'PB', 'AM', 'DF', 'PI', 'PA', 'MG', 'CE',
       'RJ', 'RN', 'MS', 'MT', 'TO', 'AL', 'GO', 'AP', 'PE', 'SC', 'MA',
       'ES', 'RO', 'AC', 'SE', 'RR'], dtype=object)

In [ ]:
dic_estados={}

for key, estado_sigla in enumerate(estados):
  dic_estados[estado_sigla]=key

In [31]:
df_semi_final_numerico['SG_UF_ESC']=df_semi_final_numerico['SG_UF_ESC'].map(dic_estados)

In [32]:
df_semi_final_numerico.drop(columns=['CO_UF_ESC'], inplace=True)

In [34]:
display(df_semi_final_numerico.head())
print("="*50)

display(df_semi_final_numerico.info())

print("="*50)

display(df_semi_final_numerico.describe())

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,N_PESSOAS_MESMA_RED,INTERNET,MEDIA_CANDIDATO,SCORE_CONSUMO,SCORE_CULT_PAIS,RENDA
NU_INSCRICAO,,,,,,,,,,,,,,,,,,,
190001867757,2,1,1,1,1,2,0,3,1.0,0,4.0,1.0,1.0,2,1,722.58,18,8.0,2245
190001692704,3,1,1,1,1,2,0,2,1.0,1,2.0,1.0,1.0,3,0,477.44,13,4.0,0
190001595660,2,1,1,2,1,2,0,2,1.0,2,2.0,1.0,1.0,3,1,498.76,7,11.0,499
190001421552,2,0,1,2,1,2,0,2,1.0,3,2.0,1.0,1.0,2,1,614.34,7,5.0,499
190001480443,3,0,2,3,4,2,0,2,1.0,4,2.0,1.0,1.0,4,1,422.28,11,6.0,1247


<class 'pandas.core.frame.DataFrame'>
Index: 880332 entries, 190001867757 to 190005863624
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   TP_FAIXA_ETARIA         880332 non-null  int64  
 1   TP_SEXO                 880332 non-null  int64  
 2   TP_ESTADO_CIVIL         880332 non-null  int64  
 3   TP_COR_RACA             880332 non-null  int64  
 4   TP_NACIONALIDADE        880332 non-null  int64  
 5   TP_ST_CONCLUSAO         880332 non-null  int64  
 6   TP_ANO_CONCLUIU         880332 non-null  int64  
 7   TP_ESCOLA               880332 non-null  int64  
 8   TP_ENSINO               880332 non-null  float64
 9   SG_UF_ESC               880332 non-null  int64  
 10  TP_DEPENDENCIA_ADM_ESC  880332 non-null  float64
 11  TP_LOCALIZACAO_ESC      880332 non-null  float64
 12  TP_SIT_FUNC_ESC         880332 non-null  float64
 13  N_PESSOAS_MESMA_RED     880332 non-null  int64  
 14  INTERNET

None

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,N_PESSOAS_MESMA_RED,INTERNET,MEDIA_CANDIDATO,SCORE_CONSUMO,SCORE_CULT_PAIS,RENDA
count,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000,880332.0,880332.0,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000,880332.000000
mean,2.687800,0.579653,0.993292,2.075308,1.032144,2.0,0.0,2.140436,1.002525,9.112913,2.303818,1.034603,1.005968,4.066861,0.789065,529.750393,15.257395,5.873259,2735.655411
std,1.076375,0.493615,0.235934,1.027651,0.220854,0.0,0.0,0.347439,0.050188,6.982482,0.813873,0.182772,0.124076,1.338404,0.407973,80.889918,9.210819,4.162783,3903.095788
min,1.000000,0.000000,0.000000,0.000000,0.000000,2.0,0.0,2.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,96.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,1.000000,1.000000,1.000000,2.0,0.0,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,3.000000,1.000000,469.260000,8.000000,2.000000,499.000000
50%,3.000000,1.000000,1.000000,2.000000,1.000000,2.0,0.0,2.000000,1.000000,9.000000,2.000000,1.000000,1.000000,4.000000,1.000000,520.740000,14.000000,6.000000,1247.000000
75%,3.000000,1.000000,1.000000,3.000000,1.000000,2.0,0.0,2.000000,1.000000,13.000000,2.000000,1.000000,1.000000,5.000000,1.000000,583.600000,20.000000,8.000000,2744.000000
max,19.000000,1.000000,4.000000,5.000000,4.000000,2.0,0.0,3.000000,2.000000,26.000000,4.000000,2.000000,4.000000,20.000000,1.000000,850.820000,79.000000,14.000000,25000.000000


### Salva em Zip para Reutilização

In [36]:
caminho_saida= r"../dados/microdados_enem_2019/dados_enem_processados_v2.csv.zip"

df_semi_final_numerico.to_csv(caminho_saida, index=True)